In [1]:
%load_ext google.cloud.bigquery

/Users/josh.carp/miniconda3/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [251]:
import collections
from dataclasses import dataclass

import branca
import folium
import geojson
import pytz
import shapely

from folium.plugins import TimestampedGeoJson

In [3]:
%%bigquery inservice
with intervals as (
  select
    routeid,
    equipmentid,
    timestamp_seconds(cast(floor(unix_seconds(timestamp(receivetimestamp)) / 3600) * 3600 as int64)) as receiveinterval,
  from `cvilledata.cat.vehicles`
  where inservice = 1
  group by routeid, equipmentid, receiveinterval
), counts as (
  select
    routeid,
    receiveinterval,
    count(equipmentid) as buses,
  from intervals
  group by routeid, receiveinterval
)
select
  counts.receiveinterval,
  counts.buses,
  routes.name,
  routes.abbr,
from counts
join `cvilledata.cat.routes` routes on counts.routeid = cast(routes.id as string)
order by receiveinterval desc

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3225/3225 [00:01<00:00, 2749.22rows/s]


In [436]:
%%bigquery vehicles
with ranked as (
    select
      *,
      -- st_buffer(st_geogpoint(lng, lat), 150) as position,
      st_geogpoint(lng, lat) as position,
      timestamp_trunc(receivetimestamp, minute) as receiveminute,
      row_number() over (partition by equipmentid, timestamp_trunc(receivetimestamp, minute) order by receivetimestamp desc) as rank
    from `cvilledata.cat.vehicles` vehicles
    join `cvilledata.cat.routes` routes on vehicles.routeid = cast(routes.id as string)
#     where routes.abbr in ('7')
    where routeid = '5'
#       and equipmentid = '103'
      and receivetimestamp >= '2022-05-11'
      and receivetimestamp < '2022-05-12'
)
select
    *
from ranked
where rank = 1

Downloading: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 929/929 [00:02<00:00, 437.90rows/s]


In [437]:
def get_position(row):
    return geojson.Feature(
        id=row.name,
        geometry=shapely.wkt.loads(row.position),
        properties={
            "icon": "circle",
            "iconstyle": {"color": row.color, "fillColor": row.color, "fillOpacity": 1},
            "times": [row.receiveminute.replace(tzinfo=pytz.timezone("US/Eastern")).timestamp() * 1000],
        },
    )
    
positions = vehicles.apply(get_position, axis=1)

In [438]:
fc = geojson.FeatureCollection(list(positions))
# fc

sd

In [439]:
m = folium.Map([38.0293, -78.4767,], zoom_start=13)

g = TimestampedGeoJson(
    dict(fc),
    auto_play=False,
    period="PT1M",
    duration="PT1S",
    max_speed=25,
    loop=False,
).add_to(m)

add_categorical_legend(m, "Routes", route_colors, route_names)

m

In [106]:
# Adapted from https://github.com/mrcagney/examples_folium/blob/develop/notebooks/categorical_legend.ipynb
def add_categorical_legend(map_, title, colors, labels):
    """
    Given a Folium map, add to it a categorical legend with the given title, colors, and corresponding labels.
    The given colors and labels will be listed in the legend from top to bottom.
    Return the resulting map.
    
    Based on `this example <http://nbviewer.jupyter.org/gist/talbertc-usgs/18f8901fc98f109f2b71156cf3ac81cd>`_.
    """
    # Error check
    if len(colors) != len(labels):
        raise ValueError("colors and labels must have the same length.")

    color_by_label = dict(zip(labels, colors))

    # Make legend HTML
    template = f"""
    {{% macro html(this, kwargs) %}}

    <!doctype html>
    <html lang="en">
    <head>
      <meta charset="utf-8">
      <meta name="viewport" content="width=device-width, initial-scale=1">
    </head>
    <body>
    <div id='maplegend' class='maplegend'>
      <div class='legend-title'>{title}</div>
      <div class='legend-scale'>
        <ul class='legend-labels'>
    """

    for label, color in color_by_label.items():
        template += f"<li><span style='background:{color}'></span>{label}</li>"

    template += """
        </ul>
      </div>
    </div>

    </body>
    </html>

    <style type='text/css'>
      .maplegend {
        position: absolute;
        z-index:9999;
        background-color: rgba(255, 255, 255, 1);
        border-radius: 5px;
        border: 2px solid #bbb;
        padding: 10px;
        font-size:12px;
        right: 10px;
        bottom: 20px;
      }
      .maplegend .legend-title {
        text-align: left;
        margin-bottom: 5px;
        font-weight: bold;
        font-size: 90%;
        }
      .maplegend .legend-scale ul {
        margin: 0;
        margin-bottom: 5px;
        padding: 0;
        float: left;
        list-style: none;
        }
      .maplegend .legend-scale ul li {
        font-size: 80%;
        list-style: none;
        margin-left: 0;
        line-height: 18px;
        margin-bottom: 2px;
        }
      .maplegend ul.legend-labels li span {
        display: block;
        float: left;
        height: 16px;
        width: 30px;
        margin-right: 5px;
        margin-left: 0;
        border: 0px solid #ccc;
        }
      .maplegend .legend-source {
        font-size: 80%;
        color: #777;
        clear: both;
        }
      .maplegend a {
        color: #777;
        }
    </style>
    {% endmacro %}
    """

    macro = branca.element.MacroElement()
    macro._template = branca.element.Template(template)
    map_.get_root().add_child(macro)

    return map_

In [133]:
routes_sorted = vehicles[["color", "name", "id"]].drop_duplicates().sort_values(["id"])
route_names = list(routes_sorted.name)
route_colors = list(routes_sorted.color)

In [162]:
vehicles.receiveTimestamp.min()

Timestamp('2022-05-11 06:17:32')

In [164]:
mm = vehicles.receiveminute.min()

In [168]:
mm.timestamp()

1652249820.0

In [171]:
mm.replace(tzinfo=pytz.timezone("US/Eastern")).timestamp()

1652264220.0

In [267]:
%%bigquery vehicles_0511
select
    *
from `cvilledata.cat.vehicles`
where receivetimestamp >= '2022-05-11'
    and receivetimestamp < '2022-05-12'
    and inservice = 1
order by receivetimestamp

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28734/28734 [00:15<00:00, 1858.56rows/s]


In [400]:
%%bigquery stop_changes
with lagged as (
    select
        *,
        lag(receivetimestamp, 1) over (partition by equipmentid order by receivetimestamp) as lastReceiveTimestamp,
        lag(nextstopid, 1) over (partition by equipmentid order by receivetimestamp) as lastNextStopID,
        lag(direction, 1) over (partition by equipmentid order by receivetimestamp) as lastDirection,
        lag(inservice, 1) over (partition by equipmentid order by receivetimestamp) as lastInService,
    from `cvilledata.cat.vehicles`
    where receivetimestamp >= '2022-05-11'
        and receivetimestamp < '2022-05-12'
)
select
    *
from lagged
where nextstopid != lastnextstopid
    and inservice = 1
    and lastinservice = 1
order by receivetimestamp

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8010/8010 [00:06<00:00, 1188.86rows/s]


In [275]:
%%time

stops = collections.defaultdict(set)
transitions = collections.defaultdict(set)
seen = set()

@dataclass(frozen=True)
class Stop:
    stop_id: str
    direction: str
        
    @classmethod
    def from_next_stop(cls, next_stop):
        return cls(stop_id=next_stop["stopID"], direction=next_stop["direction"])
    
    @classmethod
    def from_row(cls, row):
        return cls(stop_id=row.stopID, direction=row.direction)

for _, row in vehicles_0511.iterrows():
    key = (row.receiveTimestamp, row.equipmentID)
    if key in seen:
        continue
    for prev_stop, next_stop in zip(row.minutesToNextStops[:-1], row.minutesToNextStops[1:]):
        stops[row.routeID].add(Stop.from_next_stop(prev_stop))
        stops[row.routeID].add(Stop.from_next_stop(next_stop))
        transitions[row.routeID].add((Stop.from_next_stop(prev_stop), Stop.from_next_stop(next_stop)))
    seen.add(key)

CPU times: user 6.48 s, sys: 210 ms, total: 6.69 s
Wall time: 8.04 s


In [269]:
for stop in stops.keys():
    print(stop, len(stops[stop]), len(transitions[stop]))

12 48 48
2 46 46
6 67 67
11 31 31
7 41 41
8 53 53
9 39 39
1 30 30
5 51 51
3 20 20
4 40 40
10 46 46


In [401]:
def iter_stops(prev_stop, next_stop, transitions, max_stops=20):
    curr_stop = prev_stop
    int_stops = []
    for attempt in range(max_stops):
        curr_stop = transitions[curr_stop]
        int_stops.append(curr_stop)
        if curr_stop == next_stop:
            break
    return int_stops, attempt

reached = []
for row in stop_changes.itertuples():
    if row.lastDirection == row.direction and row.direction is not None:
        reached.append(
            iter_stops(
                Stop(row.lastNextStopID, row.direction),
                Stop(row.nextStopID, row.direction),
                dict(transitions[row.routeID]),
                max_stops=10,
            )
        )
    else:
        pairs = [
            (Stop(row.lastNextStopID, "Inbound"), Stop(row.nextStopID, "Inbound")),
            (Stop(row.lastNextStopID, "Inbound"), Stop(row.nextStopID, "Outbound")),
            (Stop(row.lastNextStopID, "Outbound"), Stop(row.nextStopID, "Inbound")),
            (Stop(row.lastNextStopID, "Outbound"), Stop(row.nextStopID, "Outbound")),
        ]
        attempt = None
        for pair in pairs:
            try:
                int_stops_tmp, attempt_tmp = iter_stops(pair[0], pair[1], dict(transitions[row.routeID]), 10)
                if attempt is None or attempt_tmp < attempt:
                    int_stops, attempt = int_stops_tmp, attempt_tmp
            except:
                pass
        if attempt is None:
            raise ValueError
        reached.append((int_stops, attempt))
            

In [402]:
attempts = pd.Series([attempt for _, attempt in reached])
stop_changes[attempts == 9][["routeID", "equipmentID", "nextStopID", "direction", "lastNextStopID", "lastDirection", "receiveTimestamp", "lastReceiveTimestamp"]]

,routeID,equipmentID,nextStopID,direction,lastNextStopID,lastDirection,receiveTimestamp,lastReceiveTimestamp
3131,10,312,20096,Outbound,11015,Outbound,2022-05-11 12:57:45,2022-05-11 12:57:17
4454,4,106,11494,Inbound,15887,Inbound,2022-05-11 15:25:14,2022-05-11 15:24:59
4456,4,106,15860,Inbound,11494,Inbound,2022-05-11 15:25:46,2022-05-11 15:25:14
4704,5,101,11193,Outbound,11138,Inbound,2022-05-11 15:57:41,2022-05-11 15:57:11
5260,5,101,19444,Inbound,19324,Inbound,2022-05-11 17:02:45,2022-05-11 17:02:13
5322,8,205,16680,Inbound,16596,Inbound,2022-05-11 17:10:13,2022-05-11 17:09:26
6045,5,101,19437,Outbound,17333,Outbound,2022-05-11 18:37:49,2022-05-11 18:37:10
6949,8,103,11062,Inbound,11193,Outbound,2022-05-11 20:18:49,2022-05-11 20:18:00


In [433]:
dtms = pd.Series([stops for stops, _ in reached]).apply(lambda stops: any(stop.stop_id == '17333' for stop in stops))

In [435]:
stop_changes[dtms & (stop_changes.routeID == '5')]

,nextStopExtID,aID,receiveTime,receiveTimestamp,trainID,vehicleType,nextStopPctProg,onSchedule,inService,directionAbbr,...,lng,load,routeID,lastStopExtID,equipmentID,nextStopETA,lastReceiveTimestamp,lastNextStopID,lastDirection,lastInService
420,17333,354676050582070,1652268370000,2022-05-11 07:26:10,21895,Bus,44.124020,0,1,None,...,-78.47461,0,5,11321,101,450,2022-05-11 07:25:07,11321,None,1
902,17333,354676050582070,1652271960000,2022-05-11 08:26:00,21896,Bus,41.965040,0,1,IN,...,-78.47445,0,5,11321,101,510,2022-05-11 08:24:54,11321,Inbound,1
1394,17333,354676050582070,1652275404000,2022-05-11 09:23:24,21897,Bus,28.679040,0,1,None,...,-78.47386,0,5,11321,101,570,2022-05-11 09:22:26,11321,None,1
1905,17333,354676050582070,1652278877000,2022-05-11 10:21:17,21898,Bus,44.124020,0,1,IN,...,-78.47455,0,5,11321,101,630,2022-05-11 10:20:45,11321,Inbound,1
2415,17333,354676050582070,1652282695000,2022-05-11 11:24:55,21899,Bus,29.001450,0,1,IN,...,-78.47387,0,5,11321,101,690,2022-05-11 11:24:13,11321,Inbound,1
2921,17333,354676050582070,1652286830000,2022-05-11 12:33:50,21900,Bus,30.214090,-4,1,IN,...,-78.47632,0,5,11321,101,754,2022-05-11 12:31:43,11321,Inbound,1
3412,17333,354676050582070,1652290253000,2022-05-11 13:30:53,21901,Bus,4.261932,-2,1,IN,...,-78.47385,0,5,11321,101,812,2022-05-11 13:30:15,11321,Inbound,1
3918,17333,354676050582070,1652293458000,2022-05-11 14:24:18,21902,Bus,0.000000,0,1,IN,...,-78.47425,0,5,11321,101,870,2022-05-11 14:23:49,11321,Inbound,1
4511,17333,354676050582070,1652297603000,2022-05-11 15:33:23,21903,Bus,11.044480,-5,1,IN,...,-78.47353,0,5,11321,101,935,2022-05-11 15:32:42,11321,Inbound,1
5167,17333,354676050582070,1652302247000,2022-05-11 16:50:47,21904,Bus,30.613410,-21,1,IN,...,-78.47391,0,5,11321,101,1011,2022-05-11 16:50:22,11321,Inbound,1


In [191]:
route4.minutesToNextStops[0][0]

{'routeID': '5',
 'equipmentID': '101',
 'directionAbbr': 'OUT',
 'time': '06:30AM',
 'track': 3,
 'statuscolor': '#39B139',
 'scheduleNumber': '06:30:00-16',
 'direction': 'Outbound',
 'minutes': 6,
 'timePoint': 1,
 'patternStopID': '6855',
 'stopID': '17333',
 'status': 'On Time',
 'schedule': '06:30AM',
 'blockID': '45'}

In [192]:
transitions = []

for seq in route4.minutesToNextStops:
    for stop, nextstop in zip(seq[:-1], seq[1:]):
        transitions.append((stop["direction"] + stop["stopID"], nextstop["direction"] + nextstop["stopID"]))

In [199]:
transitions_unique = set(transitions)

In [200]:
len(transitions_unique)

51

In [202]:
len(set(t[0] for t in transitions_unique))

51